# Ocean Carrier Alliances Project - Analysis and Modeling (PNREC Version)

This notebook analyses the PIERS BOL data from the [PIERS Data Project](https://github.com/epistemetrica/PIERS-Data-Project), primarily with the May 23 PNREC conference presentation in mind, but also preparing for later publication(s). Since our initial interest is the impact of alliances on domestic producers, we start by analysing the exports database. 

## Data Import and Transformation

The data come from the PIERS database and are primarily processed in the [PIERS Data Project](https://github.com/epistemetrica/PIERS-Data-Project) with specific preparations for this analysis occuring in the 'oca_data_pred.ipynb' file in this repository. 

In [17]:
#preliminaries 
import pandas as pd #v2.1.3
import numpy as np
import polars as pl #v0.20.18
import plotly_express as px #v0.4.1 
import plotly.graph_objects as go
import datetime as dt
import statsmodels.api as sm
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

#enable string cache for polars categoricals
pl.enable_string_cache()

#load data to lazyframe
exports_lf = pl.scan_parquet('data/exports/exports.parquet')
imports_lf = pl.scan_parquet('data/imports/*.parquet')

#ignore unused columns 
exports_lf = (
    exports_lf.select(
        'teus',
        #'carrier_name',
        #'carrier_scac',
        #'vessel_name',
        #'voyage_number',
        'vessel_id',
        'departure_port_code',
        'departure_port_name',
        #'coast_region', #note coast region was restricted to west coast in the data_prep nb
        'hs_code',
        #'date',
        #'dest_territory',
        #'dest_region',
        'arrival_port_code',
        'arrival_port_name',
        'direction',
        'bol_id',
        'year',
        'month',
        'lane_id',
        'lane_name',
        'unified_carrier_name',
        'unified_carrier_scac',
        'vessel_owner',
        'primary_cargo',
        'vessel_lane_pair',
        'date_departure'
    )
)
imports_lf = (
    imports_lf.select(
        'teus',
        #'date',
        #'origin_territory',
        'origin_region',
        'arrival_port_code',
        'arrival_port_name',
        'departure_port_code',
        'departure_port_name',
        #'coast_region', #note coast region was restricted to west coast in the data_prep nb
        'hs_code',
        #'carrier_name',
        #'carrier_scac',
        #'vessel_name',
        #'voyage_number',
        'vessel_id',
        'direction',
        'bol_id',
        'year',
        'month',
        'lane_id',
        'lane_name',
        'unified_carrier_name',
        'unified_carrier_scac',
        'vessel_owner',
        'primary_cargo',
        'vessel_lane_pair',
        'date_arrival'
    )
)

### Alliance Membership

Data on which carriers are part of which alliances was collected, where available, from alliance agreements filed with the Federal Maritime Commission. Where primary sources were not available, alliance membership was determined from industry reports and various media sources. 

In [18]:
#load alliance membership data from csv
alliances_df = pl.read_csv('data/misc/alliance_memberships.csv', dtypes={'scac':pl.Categorical, 'year':pl.Int32}).drop('carrier')
#alliances_df.head()

Since we are initially concerned with analysis based on whether or not a carrier was a member of an alliance, we unify the above into a single boolean column.

In [19]:
alliances_df = (
    alliances_df.with_columns(
        #set new 'alliance_member' column to 1 if the carrier is a member of any alliance
        pl.when((pl.col('2m') + pl.col('ocean') + pl.col('the'))>0)
        .then(pl.lit(1))
        .otherwise(pl.lit(0))
        .alias('alliance_member')
    )
)

#join to exports lf, renaming SCAC column for convenience
exports_lf = (
    exports_lf.rename({'unified_carrier_scac':'scac'})
    .join(alliances_df.lazy(), on=['scac', 'year'], how='left')
)

#coalesce dummy columns
exports_lf = (
    exports_lf
    .with_columns(
        pl.when(pl.col('2m')==1)
        .then(pl.lit('2m'))
        .when(pl.col('ocean')==1)
        .then(pl.lit('ocean'))
        .when(pl.col('the')==1)
        .then(pl.lit('the'))
        .otherwise(pl.lit('other'))
        .alias('alliance')
    )
)

## Exploratory Analysis

From publically available documents, we know which carriers are in which alliances over time [timeline graphic]. While much as been written in industry media on the presumed impacts of these alliances, many questions remain unanswered. [lit review?]

### Alliance Volumes Through Time

First, it is worthwhile to inspect the volumes carried by alliance members over time. 

In [37]:
df = (
    exports_lf
    #group by alliance, scac, and year
    .group_by('alliance', 'scac', 'year')
    .agg(pl.col('teus').sum())
    .collect()
)

#plot
fig = px.bar(
    df, x='year', y='teus',
    color='alliance', 
    barmode='stack', 
    text='scac', 
    title='Alliance Volumes (TEU) Over Time',
    width=900,
    height=500
    )

fig.show()

In [38]:
df = (
    exports_lf
    .filter(pl.col('year')==2023)
    #group by alliance, scac, and year
    .group_by('alliance', 'scac', 'year')
    .agg(pl.col('teus').sum())
    .collect()
)

#plot
fig = px.bar(
    df, x='year', y='teus',
    color='alliance', 
    barmode='group', 
    text='scac', 
    title='Alliance Volumes (TEU) Over Time',
    width=900,
    height=500
    )

fig.show()

In [39]:
df = (
    exports_lf
    #group by alliance, scac, and year
    .group_by('alliance', 'scac', 'year')
    .agg(pl.col('teus').sum())
    #get proportion 
    .with_columns((pl.col('teus')/pl.col('teus').sum().over('year')).alias('prop_volume'))
    .collect()
)

#plot
fig = px.bar(
    df, x='year', y='prop_volume',
    color='alliance',
    #text='scac', 
    barmode='stack',  
    title='Alliance Volumes (Proportion of Total TEU) Over Time',
    width=900,
    height=500
    )

fig.show()

In [40]:
df = (
    exports_lf
    #group by alliance and month
    .group_by('alliance', 'month')
    .agg(pl.col('teus').sum())
    .sort(by='month')
    .collect()
)

px.line(
    df, x='month', y='teus',
    color='alliance',
    title='Alliance Volumes Through Time',
    width=900,
    height=500)

In [41]:
px.line(
    df.with_columns((pl.col('teus')/pl.col('teus').sum().over('month')).alias('prop_volume')), 
    x='month', y='prop_volume',
    color='alliance',
    title='Alliance Share of Volumes Through Time',
    width=900,
    height=500)

In [44]:
df = (
    exports_lf
    #group by alliance and port
    .group_by('alliance', 'departure_port_name')
    .agg(pl.col('teus').sum())
    .collect()
)

px.bar(
    df, x='departure_port_name', y='teus',
    color='alliance',
    title='Alliance Volumes by Port',
    width=900,
    height=500)

In [45]:
df = (
    exports_lf
    #group by alliance and port
    .group_by('alliance', 'departure_port_name')
    .agg(pl.col('teus').sum())
    .with_columns((pl.col('teus')/pl.col('teus').sum().over('departure_port_name')).alias('prop_volumes'))
    .collect()
)

px.bar(
    df, x='departure_port_name', y='prop_volumes',
    color='alliance',
    title='Alliance Share of Volumes by Port',
    width=900,
    height=500)

In [46]:
df = (
    exports_lf
    #filter to seattle and tacoma
    .filter((pl.col('departure_port_name')=='SEATTLE')|(pl.col('departure_port_name')=='TACOMA'))
    #group by alliance and month
    .group_by('alliance', 'month')
    .agg(pl.col('teus').sum())
    .sort(by='month')
    .collect()
)

px.line(
    df, x='month', y='teus',
    color='alliance',
    title='Alliance Volumes Through Time at Northwest Seaport Alliance (Seattle and Tacoma))',
    width=900,
    height=500)

In [47]:
df = (
    exports_lf
    .filter(pl.col('alliance_member')==1)
    #group by alliance and port
    .group_by('departure_port_name', 'month')
    .agg(pl.col('teus').sum())
    .sort(by='month')
    .cast({'departure_port_name':pl.Utf8})
    .collect()
)

px.line(
    df, x='month', y='teus',
    color='departure_port_name',
    title='Alliance Volumes by Port',
    width=900,
    height=500)


### Cargo Sharing

Fundamentally, we expect alliances to have the most impact on cargo that is *shared* (for example, Maersk cargo carried on a Mediterranean vessel) since alliance membership likely has less of an impact on each carrier's own demand. 

[sharing through time plot(s)]


In [29]:
#add sharing columns to exports lf
exports_lf = (exports_lf.with_columns((pl.col('teus')*(1-pl.col('primary_cargo'))).alias('shared_teus')))

In [48]:
df = (
    exports_lf
    #group by alliance and month
    .group_by('alliance', 'month')
    .agg(
        pl.col('teus').sum().alias('sum_teus'),
        pl.col('shared_teus').sum().alias('sum_shared_teus')
    )
    .with_columns(
        (pl.col('sum_shared_teus')/pl.col('sum_teus')).alias('prop_shared')
    )
    .sort(by='month')
    .collect()
)

px.line(
    df, x='month', y='prop_shared',
    color='alliance',
    title='Alliance Sharing Through Time',
    width=900,
    height=500)

In [49]:
df = (
    exports_lf
    #filter to seattle and tacoma
    .filter((pl.col('departure_port_name')=='SEATTLE')|(pl.col('departure_port_name')=='TACOMA'))
    #group by alliance and month
    .group_by('alliance', 'month')
    .agg(
        pl.col('teus').sum().alias('sum_teus'),
        pl.col('shared_teus').sum().alias('sum_shared_teus')
    )
    .with_columns(
        (pl.col('sum_shared_teus')/pl.col('sum_teus')).alias('prop_shared')
    )
    .sort(by='month')
    .collect()
)

px.line(
    df, x='month', y='prop_shared',
    color='alliance',
    title='Alliance Sharing Through Time at Northwest Seaport Alliance Ports',
    width=900,
    height=500)


We also look at the relationship between the prevalence of sharing cargo and the overall size of the port, theorizing that cargo sharing may be more common at smaller ports. 

In [15]:
port_share_df = (
    exports_lf
    .group_by('departure_port_code')
    .agg(
        (pl.col('shared_teus')).sum().alias('sum_shared_teus'),
        pl.col('teus').sum().alias('sum_teus')
    )
    .with_columns((pl.col('sum_shared_teus')/pl.col('sum_teus')).alias('prop_shared'))
    .sort(by='sum_teus')
    .collect()
    .to_pandas()
)
px.scatter(port_share_df, x=port_share_df.index, y=port_share_df.prop_shared,
           color=port_share_df.departure_port_code,
           labels={'index':'Port Size Rank (ascending)'},
           title='Average % Shared Cargo by Port Size'
           )

/Users/adamwilson/miniconda3/envs/wsu/lib/python3.12/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [18]:
scipy.stats.pearsonr(port_share_df.sum_teus, port_share_df.prop_shared)

PearsonRResult(statistic=0.5545477796136669, pvalue=7.72422227465227e-05)

As it turns out, sharing cargo is more common at *larger* ports, not smaller ones. 

### Sharing between Alliance Members

One potential effect of alliances is that shared capacity formerly available to all carriers becomes utilized mostly by alliance members. Since 2017, all major carriers are in alliances with other major carriers, potentially reducing the capacity for smaller carriers to move cargo on ships owned by major carriers. 

In [19]:
share_df = (
    exports_lf
    #get shared cargo col
    .with_columns(
        (pl.col('teus')*(1-pl.col('primary_cargo'))).alias('shared_teus')
    )
    .collect()
)

In [20]:
def sharing_by_carrier(lf, scac, carrier_name=None):
    '''
    ad hoc function to inspect sharing by scac over time for a given carrier
    inputs:
        scac - str - the SCAC for the carrier of interest
        carrier_name - str - the string name of the carrier of interest for plot title
    '''
    df = (
        lf
        #get shared cargo col
        .with_columns((pl.col('teus')*(1-pl.col('primary_cargo'))).alias('shared_teus'))
        #only maersk vessels but not maersk cargo
        .filter((pl.col('vessel_owner')==scac)&(pl.col('scac')!=scac))
        #get percentages of other carriers
        .group_by('scac','month')
        .agg(pl.col('shared_teus').sum())
        .with_columns(pl.col('shared_teus').sum().over('month').alias('total_shared'))
        .with_columns((pl.col('shared_teus')/pl.col('total_shared')).alias('prop_shared'))
        #sort for plotting
        .sort(by=['month', 'scac'])
        #collect to dataframe and set dtypes
        .collect()
        #.with_columns(pl.col('month').str.to_datetime('%Y%m')) 
        .cast({'scac':pl.Utf8})
        .to_pandas()
    )
    #plot
    fig = px.bar(df, x='month', y='prop_shared', color='scac', 
                color_discrete_sequence=px.colors.qualitative.Set3,
                title=str('Proportion of shared cargo on '+carrier_name+' ships by Carrier') if carrier_name else str('Proportion of shared cargo on '+scac+' ships by Carrier'),
                labels={
                    'prop_shared':'Percentage of Shared Cargo',
                    'month':'Time'
                })
    fig.show()

In [21]:
sharing_by_carrier(exports_lf, 'MSCU', 'Med Line')

In [ ]:
sharing_by_carrier(exports_lf, 'MLSL', 'Maersk Line')

## Regression

The initial model for the project inspects the effects of carrier alliances on frequency of service. We estimate this using the following equation:

$$ S_{lct} = b X_{lct} + a_1 AM_{ct}PC_{lc} + a_2 AM_{ct} + a_3 PC_{lc} + \epsilon_{lct} $$

where:
- $l$ is the lane, i.e. the departure port and arrival port pair
- $c$ is the carrier
- $t$ is the time period, which we aggregate to months (e.g., May 2019)
- $S$ is the frequency of service, i.e. the number of voyages provided to that lane by the carrier in the given month. 
- $X$ are the correction variables
- $AM$ is an indicator of whether or not the carrier is part of an alliance
- $PC$ is an indicator of whether or not the lane was serviced by that carrier before the alliance took effect. 

Since our initial interest is the impact of alliances on domestic producers, we start by analysing the exports database. 

In [ ]:
#collect data into regular dataframe for analysis
df = (
    exports_lf
    #aggregate by lane, carrier, and month, counting unique departure dates to determine service frequency
    .group_by('lane_id', 'unified_carrier_scac', 'year', 'month')
    .agg(pl.col('date_departure').unique().count().alias('service_freq'))
    #rename carrier code column
    .rename({'unified_carrier_scac':'scac'})
    #cast data types
    .cast({'service_freq':pl.Int16})
    .with_columns(pl.col('month').str.to_datetime('%Y%m'))
    .collect()
)

#display
display(df.describe())
df.head()

In [ ]:

#join alliance membership data into main df
df = (
    df.join(alliances_df, on=['scac', 'year'], how='left')
    #set missing alliance membership values to null 
    .with_columns(pl.col('alliance_member').fill_null(0))
)

In [ ]:
df.head()

### Identifying whether a lane was serviced by that carrier before the alliance took effect

In [ ]:
df= (
    #get total voyages prior to joining an alliance for each lane and carrier
    df.with_columns(
        pl.when(pl.col('alliance_member')==0)
        .then(pl.col('service_freq').sum().over('lane_id', 'scac'))
        .otherwise(pl.lit(0))
        .alias('service_prior')
    )
    #simplify to 1 if any service prior to alliance, 0 otherwise
    .with_columns(
        pl.when(pl.col('service_prior').sum()>0)
        .then(pl.lit(1))
        .otherwise(pl.lit(0))
        .over('lane_id', 'scac')
        .alias('service_prior')
    )
)
df.head()

## Simple OLS Regression

Leaving out any correction variables for the moment, we estimate a simple OLS regression model on the equation above. 

In [ ]:
#set dependent and independent variables
Y = df.select('service_freq').to_pandas()
X = (
    df.select('alliance_member', 'service_prior')
    .with_columns((pl.col('alliance_member')*pl.col('service_prior')).alias('alliance*prior'))
    .to_pandas()
)
#add intercept
X = sm.add_constant(X)
#instantiate model
model = sm.OLS(Y,X)
#run regression
results = model.fit()

#display
results.summary()


In [ ]:
#obtain moments for residuals
mu, std = scipy.stats.norm.fit(results.resid)

#plot resid distribution
fig, ax = plt.subplots()
sns.histplot(x=results.resid, ax=ax, stat='density', linewidth=0, kde=True)
ax.set(title='Distribution of model_OLS residuals', xlabel='residual')

#add corresponding normal curve
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = scipy.stats.norm.pdf(x,mu,std)
sns.lineplot(x=x, y=p, color='red', ax=ax)
plt.show()

In [ ]:
sm.qqplot(results.resid, line='45', fit=True);

### Limitations

1. It's broke. 
2. The simplistic method of identifying when a carrier had previously serviced that lane results in a uninteresting alliance*prior interaction term. We need to decide on a better means of identification. 
3. Complete alliance membership data only goes back to 2017; we need to update that table. 